In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import scipy.stats as stats

In [87]:
# sns.get_dataset_names()
df = sns.load_dataset("attention")
df = df.iloc[:,1:]
df.head()

,subject,attention,solutions,score
0,1,divided,1,2.0
1,2,divided,1,3.0
2,3,divided,1,3.0
3,4,divided,1,5.0
4,5,divided,1,4.0


In [88]:
df.dropna(inplace=True)

In [89]:
df.sample()

,subject,attention,solutions,score
20,1,divided,2,4.0


In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   subject    60 non-null     int64  
 1   attention  60 non-null     object 
 2   solutions  60 non-null     int64  
 3   score      60 non-null     float64
dtypes: float64(1), int64(2), object(1)
memory usage: 2.0+ KB


In [91]:
df.describe()

,subject,solutions,score
count,60.000000,60.000000,60.000000
mean,10.500000,2.000000,5.958333
std,5.814943,0.823387,1.621601
min,1.000000,1.000000,2.000000
25%,5.750000,1.000000,5.000000
50%,10.500000,2.000000,6.000000
75%,15.250000,3.000000,7.000000
max,20.000000,3.000000,9.000000


In [95]:
y = df["score"]
x = df[["subject","solutions"]]
x.sample(5)

,subject,solutions
19,20,1
11,12,1
30,11,2
55,16,3
21,2,2


<h1>Before box Cox Tranformation</h1>

In [96]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [97]:
X_train,X_test,Y_train,Y_test = train_test_split(x,y,test_size=0.2)

In [126]:
model = LinearRegression()
model = model.fit(X_train,Y_train)

In [127]:
y_pred = model.predict(X_test)

In [128]:
y_pred

array([7.04689046, 7.30502389, 6.37764471, 4.41773181, 4.95791084,
       7.152045  , 5.60324442, 6.14342346, 6.63577814, 4.82884413,
       6.89391157, 7.69222404])

In [129]:
Y_test

53    7.0
55    7.0
33    8.0
3     5.0
22    5.0
39    6.0
27    7.0
46    6.0
35    8.0
21    4.0
37    8.0
58    6.0
Name: score, dtype: float64

In [130]:
model.intercept_

3.232219197967701

In [131]:
Y_test

53    7.0
55    7.0
33    8.0
3     5.0
22    5.0
39    6.0
27    7.0
46    6.0
35    8.0
21    4.0
37    8.0
58    6.0
Name: score, dtype: float64

In [132]:
mse = mean_squared_error(Y_test, y_pred)

In [133]:
mse

1.0834937097054698

In [106]:
df.sample(5)

,subject,attention,solutions,score
50,11,focused,3,6.0
34,15,focused,2,8.0
41,2,divided,3,5.0
39,20,focused,2,6.0
11,12,focused,1,8.0


<h1>After Box Cox Transformation</h1>

In [107]:
from sklearn.preprocessing import PowerTransformer

In [113]:
tf = PowerTransformer(method="box-cox")

In [118]:
X_train_transformed = tf.fit_transform(X_train)
X_test_transformed = tf.fit_transform(X_test)

In [119]:
model = model.fit(X_train_transformed,Y_train)
transformed_yPred = model.predict(X_test_transformed)

In [124]:
mse = mean_squared_error(transformed_yPred, y_pred)
mse

0.19307878481049837

In [121]:
transformed_yPred

array([6.84306917, 7.06612345, 5.89060725, 3.79128101, 4.54114975,
       6.54828961, 5.18945526, 6.01866687, 6.11366153, 4.39659978,
       6.33272027, 7.39337606])